In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import hash_model as image_hash_model
import pickle
import time
import numpy as np

top_k = 1000
batch_size = 10
epochs = 100
lr = 0.001 #0.05
weight_decay = 10 ** -5
lambda1 = 0.01
hash_bits = 64 # 假设哈希编码的位数
model_name = "vgg11"
device = torch.device("cuda")

# 1. 加载CIFAR-10数据集
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


model = image_hash_model.HASH_Net(model_name, hash_bits)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.3, last_epoch=-1)

# 3. 加载哈希编码
with open('../labels/64_cifar10_10_class.pkl', 'rb') as f:
    label_hash_codes= torch.load(f)
label_hash_codes.to(device)

/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.],
        [ 1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
          1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,
         -1., -1.,  1.,  1., -1

In [7]:
import time
import numpy as np
import torch
import torch.nn as nn
from cac import test_accuracy
from condition import add_label_noise

noise_rate=0.1
# 初始化
start_time = time.time()
total_loss = []
accuracy_list = []  # 存储每次epoch的准确率
best_accuracy = 0.0  # 存储最高准确率

for epoch in range(epochs):  # 循环遍历数据集多次
    scheduler.step()
    if epoch % 5 == 0:
        accuracy = test_accuracy(model, testloader, label_hash_codes,device)
        print(f'Epoch {epoch}, Test Accuracy: {accuracy}%')
        accuracy_list.append(accuracy)  # 添加当前准确率到列表
        
        # 检查是否是最高准确率
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            # 保存模型
            torch.save(model.state_dict(), './model/nr_'+str(noise_rate)+'_model.pth')
            print("Model saved with accuracy: {:.2f}%".format(best_accuracy))
    
    for iter, traindata in enumerate(trainloader, 0):
        inputs, labels = traindata
        labels = add_label_noise(labels.numpy(), noise_rate=noise_rate, num_classes=10)
        labels = torch.from_numpy(labels)
        
        inputs = inputs.to(device)
        inputs = model(inputs)
        cat_codes = label_hash_codes[labels].to(device) 
        criterion = nn.BCELoss().to(device)
        center_loss = criterion(0.5*(inputs+1),0.5*(cat_codes+1))

        Q_loss = torch.mean((torch.abs(inputs)-1.0)**2)
        loss = center_loss + lambda1*Q_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.data.to(device).numpy)
        if iter % 500 == 0:
            end_time1 = time.time()
            print('epoch: %d, lr: %.5f iter_num: %d, time: %.3f, loss: %.3f' % 
                  (epoch, lr, iter, (end_time1-start_time), loss))
        
end_time = time.time()
epoch_loss = np.mean(total_loss)
print('Finished Training')

# 打印所有epoch的准确率
print("Epoch accuracies:", accuracy_list)


Epoch 0, Test Accuracy: 84.36%
Model saved with accuracy: 84.36%
epoch: 0, lr: 0.00100 iter_num: 0, time: 17.771, loss: 0.275
epoch: 0, lr: 0.00100 iter_num: 100, time: 19.755, loss: 0.396
epoch: 0, lr: 0.00100 iter_num: 200, time: 21.785, loss: 0.457
epoch: 0, lr: 0.00100 iter_num: 300, time: 23.943, loss: 0.334
epoch: 0, lr: 0.00100 iter_num: 400, time: 26.153, loss: 0.251
epoch: 0, lr: 0.00100 iter_num: 500, time: 28.401, loss: 0.370
epoch: 0, lr: 0.00100 iter_num: 600, time: 30.273, loss: 0.382
epoch: 0, lr: 0.00100 iter_num: 700, time: 32.363, loss: 0.249
epoch: 0, lr: 0.00100 iter_num: 800, time: 34.335, loss: 0.301
epoch: 0, lr: 0.00100 iter_num: 900, time: 36.405, loss: 0.522
epoch: 0, lr: 0.00100 iter_num: 1000, time: 38.368, loss: 0.356
epoch: 0, lr: 0.00100 iter_num: 1100, time: 40.413, loss: 0.320
epoch: 0, lr: 0.00100 iter_num: 1200, time: 42.718, loss: 0.360
epoch: 0, lr: 0.00100 iter_num: 1300, time: 44.959, loss: 0.304
epoch: 0, lr: 0.00100 iter_num: 1400, time: 47.096,

In [ ]:
screen -S exp 